### LEGAL ESSAY GPT

The codebase for the thesis 'Persuasive Legal Writing Using Large Language Models'
The code below generates long form writing (such as essays) based on the prompt details in the input file.

In [16]:
# Imports

import openai, json, datetime, time, pickle
import pandas as pd
from tqdm import tqdm
model = 'gpt-4'

#### OUTLINE    

In [15]:
# The schema to produce the essay outline (for use in OpenAI GPT-4 'functions' call):

outline_schema = {
  "type": "object",
  "properties": {
    "sections": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "title": {
              "type": "string",
              "description": "A title for the section, including 'Introduction' and 'Conclusion'."
          },
          "content": {
              "type": "string",
              "description": "The content of that section in three short sentences. Be concise. Use an instructive tone. Use the imperative form of any verbs."
          },
        },
        "required": ["title", "summary"]
      }
    }
  },
  "required": ["sections"]
}

In [14]:
# Method to get essay outline:

def get_outline(outline_schema, background, persona, task_generate_outline):

    print("Getting outline ...", end = '')

    messages=[
        {"role": "system", "content": background},
        {"role": "user", "content": persona + task_generate_outline}
    ]

    print(messages)

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        functions=[{"name": "outline", "parameters": outline_schema}],
        function_call={"name": "outline"},
        temperature=0,
    )

    outline = completion.choices[0].message.function_call.arguments
    outline = json.loads(outline)

    print(" done.")

    return outline


#### ESSAYS

In [13]:
# Method to produce essay using the Recursive method:

def get_essay_recursive(outline, background, persona, task_write_section, section_length, additional_instruct_essay):

    print("Getting essay using Method 1 (Recursive): ... ")

    outline_string = ""
    for _, so in enumerate(outline['sections']):
        outline_string += "Title: " + so['title'] + ". Content: " + so['content'] + "\n"
    outline_string += "\n"

    total_tokens = 0
    essay = ""

    messages=[{"role": "system", "content": background}]

    for i, so in enumerate(tqdm(outline['sections'])):

        if i == 0:
            specific_instruction = " In accordance with the above instructions, write the following section only: " + so['title'] + ". "
            messages.append({"role": "user", "content": persona + task_write_section + outline_string + section_length + additional_instruct_essay + specific_instruction})
        else:
            nudge = " In accordance with the earlier instructions, continue the essay by writing the next section only: " + so['title'] + ". "
            messages.append({"role": "user", "content": nudge})

        print(messages)

        completion = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0
        )
        total_tokens += completion.usage.total_tokens
        section = completion.choices[0].message.content

        messages.append({"role": "system", "content": section})

        essay += so['title'] + "\n\n"
        essay += section + "\n\n"

    print("...done.")

    return essay, total_tokens


In [12]:
# Method to produce the essay using the Accordion method:

def get_essay_accordion(outline, background, persona, task_write_section, section_length, additional_instruct_essay):

    print("Getting essay using Method 2 (Accordion): ... ")

    outline_string = ""
    for _, so in enumerate(outline['sections']):
        outline_string += "Title: " + so['title'] + ". Content: " + so['content'] + "\n"
    outline_string += "\n"

    total_tokens = 0
    essay = ""

    for _, so in enumerate(tqdm(outline['sections'])):

        specific_instruction = " In accordance with the above instructions, write the following section only: " + so['title'] + ". "

        messages=[
            {"role": "system", "content": background},
            {"role": "user", "content": persona + task_write_section + outline_string + section_length + additional_instruct_essay + specific_instruction}
        ]

        print(messages)

        completion = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=0
        )
        total_tokens += completion.usage.total_tokens
        section = completion.choices[0].message.content

        essay += so['title'] + "\n\n"
        essay += section + "\n\n"

    print("...done.")

    return essay, total_tokens


#### GENERATE ALL

In [11]:
# Method to generate all essays:
# (Note the term 'pair' is used to refer to a 'pair' of essays, both based on the same outline but generated using different methods of Accordion and Recursive)

def generate_essays(pair_data, target_pairs=None):

    background = "You are writing part of an essay about legal theory as part of a graduate law school class in Australia. Legal theory focuses on theoretical and moral arguments and theorists, rather than empirical evidence.\n"

    for i, pair in pair_data.iterrows():

        print("===============================")

        if target_pairs != None and i not in target_pairs:
            print("\nSkipping Pair " + str(i) + " (not in target_pair list.)")
            print("-------------------------------")
            continue

        print("Generating essays for Pair " + str(pair['pair']) + " of " + str(len(pair_data)) + ":\n")

        # Pair variables:
        capability = pair['capability']
        style = pair['style']
        no_sections = pair['no_sections']
        thesis = pair['thesis']
        references = pair['references']
        topic = pair['topic']
        section_length = pair['section_length']
        additional_instruct_outline = pair['additional_instruct_outline']
        additional_instruct_essay = pair['additional_instruct_essay']

        # Persona:
        persona = capability + style

        # Prompt to generate outline:
        task_generate_outline = " Your specific task is to create a summary outline for the following essay topic. The outline should have an introduction, " + no_sections + " body sections, and a conclusion. Your stance should be to " + thesis + " with the essay topic. The introduction should assert that stance. Plan the body sections so they support that stance. Each body section should focus on one idea only, so that it can be discussed in a few paragraphs. The conclusion should reinforce the stance and what has been discussed in the body. For each section, provide information in these fields: \n'title': A title for the section, including an 'Introduction' and a 'Conclusion'; \n'content': The content of that section, in three short sentences. Be concise. Use an instructive tone. Use the imperative form of any verbs. \n" + references + additional_instruct_outline + " The essay topic is: '" + topic + "' \n"

        # Prompt to write essay section:
        task_write_section = " Your specific task is to write one section of an essay. Write that section as if it were being inserted directly into the whole essay. The outline of the entire essay is as follows: \n\n"

        print()

        # Step 1 is to produce an outline:
        # Get Outline for pair:
        outline = get_outline(outline_schema, background, persona, task_generate_outline)
        pair_data.at[i,'outline'] = outline
        now_time = datetime.datetime.now().strftime("%d_%m-%H_%M")
        outline_filename = "Pair_" + str(pair['pair']) + "_Outline_" + now_time + ".txt"
        with open('./output/' + outline_filename, 'w', errors='ignore') as f:
            f.write(json.dumps(outline, indent=4))
        print("Outline saved as: ", outline_filename)

        print()

        # Step 2 is to produce an essay, based on the outline, using the Recursive method:
        # Get Essay using Method 1 (Recursive):
        essay_recursive, tokens_recursive = get_essay_recursive(outline, background, persona, task_write_section, section_length, additional_instruct_essay)
        pair_data.at[i,'essay_recursive'] = essay_recursive
        pair_data.at[i,'tokens_recursive'] = tokens_recursive
        print("Total tokens using recursive: ", tokens_recursive)
        now_time = datetime.datetime.now().strftime("%d_%m-%H_%M")
        essay_filename = "Pair_" + str(pair['pair']) + "_Recursive_" + now_time + ".txt"
        with open('./output/' + essay_filename, 'w', encoding='utf-8', errors='ignore') as f:
            f.write(essay_recursive)
        print("Recursive essay saved as: ", essay_filename)

        # Step 3 is to produce an essay, based on the outline, using the Accordion method:
        # Get Essay using Method 2 (Accordion):
        essay_accordion, tokens_accordion = get_essay_accordion(outline, background, persona, task_write_section, section_length, additional_instruct_essay)
        pair_data.at[i,'essay_accordion'] = essay_accordion
        pair_data.at[i,'tokens_accordion'] = tokens_accordion
        print("Total tokens using accordion: ", tokens_accordion)
        now_time = datetime.datetime.now().strftime("%d_%m-%H_%M")
        essay_filename = "Pair_" + str(pair['pair']) + "_Accordion_" + now_time + ".txt"
        with open('./output/' + essay_filename, 'w', encoding='utf-8', errors='ignore') as f:
            f.write(essay_accordion)
        print("Accordion essay saved as: ", essay_filename)

        time.sleep(0.5)

        # The output details are saved as a pickle file:
        with open('./output/pair_data.pkl', 'wb') as f:
            pickle.dump(pair_data, f)

        time.sleep(0.5)

        print("-------------------------------\n")
        print()


#### CALL

In [18]:
# Example essay prompts are provided in the csv:
prompts = pd.read_csv('./prompts.csv')

# An OpenAI API key is required:
openai.api_key = input("Enter your OpenAI API key:")

# To limit the essays which are produced, limit the target pairs:
target_pairs = [0,1]

# Generate essays:
generate_essays(prompts, target_pairs)